In [1]:
import os
import torch
import numpy as np
from glob import glob
from tqdm.auto import tqdm
from sklearn.linear_model import LogisticRegression
import pandas as pd
from utils.store_data import ReadData
import numpy as np
import torch
from utils.arguments import Arguments
from tqdm import tqdm
from sklearn.decomposition import PCA
from linear_probe import get_linear_acc
import random
import os
from sklearn.metrics import roc_auc_score,accuracy_score,roc_curve

In [2]:
data_reader=ReadData('/root/autodl-fs/hallscope/pope/popular/train')
data=data_reader.read_all()
random.shuffle(data)

In [3]:
df=pd.DataFrame(data)

In [4]:
df.head()

,img_path,question,label,question_id,category,most_likely
0,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a cell phone in the image?\nAnswer t...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(5689), tensor(5690), tensor(5691), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
1,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a sink in the image?\nAnswer the que...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(6969), tensor(6970), tensor(6971), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
2,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a bench in the image?\nAnswer the qu...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(13929), tensor(13930), tensor(13931), ...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
3,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a pizza in the image?\nAnswer the qu...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(10513), tensor(10514), tensor(10515), ...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...
4,[/root/autodl-tmp/coco_images/train/COCO_train...,[Is there a bed in the image?\nAnswer the ques...,"[1, 0, 1, 0, 1, 0, 1, 0]","[tensor(1457), tensor(1458), tensor(1459), ten...","[popular, popular, popular, popular, popular, ...",{'embedding': [[[ 0.00389099 0.00149536 0.00...


In [5]:
embeddings=df['most_likely'].apply(lambda x: x['embedding']).to_list()

In [6]:
embeddings=np.concatenate(embeddings,axis=0)

In [7]:
embeddings.shape

(20000, 33, 4096)

In [8]:
gt_label=df['label'].to_list()
gt_label=np.concatenate(gt_label,axis=0)

In [9]:
gt_label.shape

(20000,)

In [10]:
responses=df['most_likely'].apply(lambda x: x['response']).to_list()
resps=[]
for x in responses:
    resps+=x

In [22]:
is_hall=[]
for i,x in enumerate(resps):
    if x.lower()=='no' and gt_label[i]==0:
        is_hall.append(1)
    elif x.lower()=='yes' and gt_label[i]==1:
        is_hall.append(1)
    else:
        is_hall.append(0)
is_hall=np.array(is_hall)

In [23]:
is_hall.shape

(20000,)

In [24]:
print(f'There are {len(is_hall)} responses. {sum(is_hall==0)} answer correctly and {sum(is_hall==1)} answer wrongly')

There are 20000 responses. 2563 answer correctly and 17437 answer wrongly


In [25]:
length=len(is_hall)
train_rate=0.8

all_indices = np.random.permutation(length)
train_val_idxs = all_indices[:int(
    train_rate * length)]  # trainset and validation index

test_idxs=all_indices[int(
    train_rate * length):]  #test index

validset_len = 1000
# exclude validation samples.
train_idxs = train_val_idxs[:len(
    train_val_idxs) - validset_len]  # trainset index
val_idxs = train_val_idxs[len(
    train_val_idxs) - validset_len:]  # validation index
hall_label_test = []
hall_label_wild = []
hall_label_val = []

'''get testset, wildset and valset. The valset is used for determining the hype-parameters'''
hall_label_test = is_hall[test_idxs]
hall_label_wild = is_hall[train_idxs]
hall_label_val = is_hall[val_idxs]

print(f'trainset length:{len(hall_label_wild)}')
print(f'validset length:{len(hall_label_val)}')
print(f'testset length:{len(hall_label_test)}')
print(f'trainset postive: {sum(hall_label_wild==1)} negative: {sum(hall_label_wild==0)}')
print(f'valset postive: {sum(hall_label_val==1)} negative: {sum(hall_label_val==0)}')
print(
    f'testset postive: {sum(hall_label_test==1)} negative: {sum(hall_label_test==0)}')

trainset length:15000
validset length:1000
testset length:4000
trainset postive: 13057 negative: 1943
valset postive: 871 negative: 129
testset postive: 3509 negative: 491


In [26]:
all_embeddings = embeddings
all_embeddings.shape

(20000, 33, 4096)

In [27]:
embedding_train = all_embeddings[train_idxs]
embedding_val = all_embeddings[val_idxs]
embedding_test = all_embeddings[test_idxs]

In [28]:
%load_ext autoreload
%autoreload 2
from ml_utils.grid_search import GridSearch
from ml_utils.PCA_discriminator import PCADiscriminator
from ml_utils.metrics import auroc,get_best_split_from_scores

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Grid Search for Best 'n_components' & 'layer'

In [29]:
def evaluator(n_components,i_layer,X,y):
    discriminator=PCADiscriminator(n_components,X[:,i_layer,:])
    scores=discriminator.get_score()
    return auroc(scores,y)

In [ ]:
# graid search for best hyper-parameters on validation set
grid={
    'n_components':range(1,12),
    'layer':range(embedding_val.shape[1])
}
grid_search=GridSearch(evaluator,grid,embedding_val,hall_label_val)
best_paras=grid_search.search()

Grid Searching for best n_components,layer


  0%|          | 0/11 [00:00<?, ?it/s]

found best n_components:1,layer:0 Score: 0.5
found best n_components:1,layer:1 Score: 0.5198070470545305


/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var


found best n_components:1,layer:6 Score: 0.5241235682054842
found best n_components:1,layer:7 Score: 0.5576678325723796
found best n_components:1,layer:8 Score: 0.6056746678058722
found best n_components:1,layer:15 Score: 0.771473580220543
found best n_components:1,layer:16 Score: 0.8313441735864506
found best n_components:1,layer:17 Score: 0.8401997169786131


  9%|▉         | 1/11 [00:01<00:16,  1.61s/it]/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
 18%|█▊        | 2/11 [00:03<00:14,  1.63s/it]/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
 27%|██▋       | 3/11 [00:05<00:13,  1.75s/it]/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ratio_ = self.explained_variance_ / total_var
 36%|███▋      | 4/11 [00:07<00:12,  1.84s/it]/root/miniconda3/envs/dl3.9/lib/python3.9/site-packages/sklearn/decomposition/_pca.py:794: RuntimeWarning: invalid value encountered in divide
  self.explained_variance_ra

: 

In [20]:
best_paras

{'n_components': 1, 'layer': 15, 'best_score': np.float64(0.8727180527383367)}

In [21]:
best_layer=best_paras['layer']

View Results on Training set and Test set

In [22]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_val[:,best_layer,:])
best_split=discriminator.get_best_split(hall_label_val)

In [23]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_train[:,best_layer,:])
scores=discriminator.get_score()
y_train=(scores>best_split)
print(f'train set acc: {accuracy_score(hall_label_wild,y_train)} AUROC:{roc_auc_score(hall_label_wild,scores)}')

train set acc: 0.8764666666666666 AUROC:0.8532297304359102


In [24]:
discriminator=PCADiscriminator(best_paras['n_components'],embedding_test[:,best_layer,:])
scores=discriminator.get_score()
preds=(scores>best_split)
print(f'test set acc: {accuracy_score(hall_label_test,preds)} AUROC:{roc_auc_score(hall_label_test,scores)}')

test set acc: 0.8635 AUROC:0.8502762849932661


Train Linear Prob to Get Better Results

In [25]:
X_train=embedding_train[:, best_layer,:]
y_train=y_train

X_test=embedding_test[:, best_layer,:]
y_test=hall_label_test

In [26]:
best_acc, final_acc, (
            clf, best_state, best_preds, preds, labels_val), losses_train = get_linear_acc(
            X_train,
            y_train,
           X_train,
            y_train,
            2, epochs=50,
            print_ret=True,
            batch_size=512,
            cosine=True,
            nonlinear=True,
            learning_rate=0.05,
            weight_decay=0.0003)

In [27]:
clf.eval()
output = clf(torch.from_numpy(X_test).to(torch.float32).cuda())
scores_test = torch.sigmoid(output).cpu().data.numpy()
pred_test=scores_test>0.5
print(f'test set acc: {accuracy_score(y_test,pred_test)} AUROC:{roc_auc_score(y_test,scores_test)}')

test set acc: 0.86 AUROC:0.7633028674538107
